# **Manual Labeling with Label Studio**

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
import cv2

In [8]:
# setting the base directory so we dont mess with the original data
basedir = '/media/parker/Data_Drive/400_manual_label'
base_image_dir = os.path.join(basedir, 'images')
base_label_dir = os.path.join(basedir, 'labels')

# new dir for images
new_dir = '/media/parker/Data_Drive/400_manual_label_aug/'
new_image_dir = os.path.join(new_dir, 'images')
new_label_dir = os.path.join(new_dir, 'labels')


# checking to see if the base directories exist
if not os.path.exists(base_image_dir):
    raise ValueError(f'Base image directory does not exist: {base_image_dir}')
if not os.path.exists(base_label_dir):
    raise ValueError(f'Base label directory does not exist: {base_label_dir}')
if not os.path.exists(new_image_dir):
    raise ValueError(f'New image directory does not exist: {new_image_dir}')
if not os.path.exists(new_label_dir):
    raise ValueError(f'New label directory does not exist: {new_label_dir}')

print(f'Base image directory: {base_image_dir}')
print(f'Base label directory: {base_label_dir}')
print(f'Augmented image directory: {new_image_dir}')
print(f'Augmented label directory: {new_label_dir}')


Base image directory: /media/parker/Data_Drive/400_manual_label/images
Base label directory: /media/parker/Data_Drive/400_manual_label/labels
Augmented image directory: /media/parker/Data_Drive/400_manual_label_aug/images
Augmented label directory: /media/parker/Data_Drive/400_manual_label_aug/labels


In [9]:
image_files = [os.path.splitext(f)[0] for f in os.listdir(new_image_dir) if f.endswith('.png')]
label_files = [os.path.splitext(f)[0] for f in os.listdir(new_label_dir) if f.endswith('.txt')]

unmatched_images = [f + '.png' for f in image_files if f not in label_files]

if unmatched_images:
    print("\nThese image files do NOT have matching label files:")
    for file in unmatched_images:
        print(f" - {file}")
else:
    print("\n All PNG images have matching TXT label files.")


 All PNG images have matching TXT label files.


In [10]:
# rotate the cords
def rotate_coords(coords, angle):
    coords = np.array(coords).reshape(-1, 2)
    if angle == 90:
        coords = np.stack([1 - coords[:, 1], coords[:, 0]], axis=1)
    elif angle == 180:
        coords = 1 - coords
    elif angle == 270:
        coords = np.stack([coords[:, 1], 1 - coords[:, 0]], axis=1)
    return coords.flatten().tolist()

In [11]:
image_files = [f for f in os.listdir(new_image_dir) if f.endswith('.png')]

print(f"found {len(image_files)} images to process...\n")

for i, filename in enumerate(image_files, 1):
    base_name = os.path.splitext(filename)[0]
    image_path = os.path.join(new_image_dir, filename)
    label_path = os.path.join(new_label_dir, base_name + '.txt')

    # skip if label file does not exist
    if not os.path.exists(label_path):
        print(f"[{i}/{len(image_files)}] Skipping {filename} (label missing)")
        continue

    # load image and label file
    image = cv2.imread(image_path)
    with open(label_path, 'r') as f:
        lines = f.readlines()


    # rotate and save
    for angle in [90, 180, 270]:
        rotated_image = cv2.rotate(image, {
            90: cv2.ROTATE_90_CLOCKWISE,
            180: cv2.ROTATE_180,
            270: cv2.ROTATE_90_COUNTERCLOCKWISE
        }[angle])

        new_image_name = f"{base_name}_rot{angle}.png"
        new_image_path = os.path.join(new_image_dir, new_image_name)
        cv2.imwrite(new_image_path, rotated_image)

        # transform label coordinates
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            class_id = parts[0]
            coords = list(map(float, parts[1:]))
            rotated_coords = rotate_coords(coords, angle)
            new_line = f"{class_id} " + " ".join(f"{c:.16f}" for c in rotated_coords)
            new_lines.append(new_line)

        new_label_name = f"{base_name}_rot{angle}.txt"
        new_label_path = os.path.join(new_label_dir, new_label_name)
        with open(new_label_path, 'w') as f:
            f.write("\n".join(new_lines))

    print(f"[{i}/{len(image_files)}]  {filename} processed")

print(" all images augmented successfully")

found 430 images to process...

[1/430]  000f0ded-image_0270.png processed
[2/430]  0056e591-image_0351.png processed
[3/430]  00b2e674-image_0020.png processed
[4/430]  00d033db-image_0325.png processed
[5/430]  01319069-image_0282.png processed
[6/430]  01471794-image_0360.png processed
[7/430]  01bce3f0-image_0274.png processed
[8/430]  02cbece9-image_0326.png processed
[9/430]  036262f4-image_0231.png processed
[10/430]  03a87745-image_0312.png processed
[11/430]  04a52baa-image_0108.png processed
[12/430]  04d8e746-image_0387.png processed
[13/430]  04ec2875-image_0375.png processed
[14/430]  055cade5-image_0400.png processed
[15/430]  057c31aa-image_0216.png processed
[16/430]  058658e0-image_0283.png processed
[17/430]  07a86152-image_0138.png processed
[18/430]  08e4458a-image_0404.png processed
[19/430]  0a06bd94-image_0209.png processed
[20/430]  0c7e4058-image_0086.png processed
[21/430]  0cb73b23-image_0328.png processed
[22/430]  0debe134-image_0229.png processed
[23/430] 